# Connect 4

In [2]:
from collections import Counter
import numpy as np
from IPython.display import clear_output

In [3]:
NUM_COLUMNS = 7
COLUMN_HEIGHT = 6
FOUR = 4

# Board can be initiatilized with `board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)`
# Notez Bien: Connect 4 "columns" are actually NumPy "rows"

## Basic Functions

In [4]:
def valid_moves(board):
    """Returns columns where a disc may be played"""
    return [n for n in range(NUM_COLUMNS) if board[n, COLUMN_HEIGHT - 1] == 0]


def play(board, column, player):
    """Updates `board` as `player` drops a disc in `column`"""
    (index,) = next((i for i, v in np.ndenumerate(board[column]) if v == 0))
    board[column, index] = player
    return board


def take_back(board, column):
    """Updates `board` removing top disc from `column`"""
    (index,) = [i for i, v in np.ndenumerate(board[column]) if v != 0][-1]
    board[column, index] = 0


def four_in_a_row(board, player):
    """Checks if `player` has a 4-piece line"""
    return (
        any(
            all(board[c, r] == player)
            for c in range(NUM_COLUMNS)
            for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1))
        )
        or any(
            all(board[c, r] == player)
            for r in range(COLUMN_HEIGHT)
            for c in (list(range(n, n + FOUR)) for n in range(NUM_COLUMNS - FOUR + 1))
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co, co + FOUR))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co + FOUR - 1, co - 1, -1))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
    )
    
def display (board):
    """Display the board like the game"""
    nb = board.T[::-1]
    for r in range(nb.shape[0]):
        for c in range(nb.shape[1]):
            if (nb[r, c] == 1):
                print("X", end=" ")
            elif (nb[r, c] == -1):
                print("O", end=" ")
            elif (nb[r, c] == 0):
                print("-", end= " ")
        print()

## Montecarlo Evaluation

In [5]:
def _mc(board, player):
    p = -player
    while valid_moves(board):
        p = -p
        c = np.random.choice(valid_moves(board))
        play(board, c, p)
        if four_in_a_row(board, p):
            return p
    return 0


def montecarlo(board, player):
    montecarlo_samples = 50
    cnt = Counter(_mc(np.copy(board), player) for _ in range(montecarlo_samples))
    return (cnt[1] - cnt[-1]) / montecarlo_samples

def minmax(board, player, level, depth):
    if (four_in_a_row(board, player)):
        return None, player
    possible = valid_moves(board)
    if (not possible):
        return None, 0
    if (level == depth):
        return None, player*montecarlo(board, player)
    evaluations = list()
    for ply in possible:
        new_board = np.copy(board)
        play(new_board, ply, player)
        _, val = minmax(new_board, -player, level+1, depth)
        evaluations.append((ply, -val))
    return max(evaluations, key=lambda k: k[1])


## Example

In [6]:
board = board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
print("Initial state of the game:")
display(board)
player = 1

while(not four_in_a_row(board, player)):
    best_ply, eval = minmax(board, player, level=0, depth=2)
    clear_output(wait=True)
    print(f"The best ply for player {player} is: {best_ply} with a percentage of {eval}")
    play(board, best_ply, player)
    display(board)
    if (four_in_a_row(board, player)):
        print(f"The player {player} won the game!")
        break
    player = -player





The best ply for player 1 is: 0 with a percentage of 1
- O O X - O X 
- O X X - X X 
- O O X - O O 
- X O O - O X 
X X X X - X O 
O O X O X X O 
The player 1 won the game!
